<h1> Chapter 10- Map & Hash </h1>


<h3> 10.1 Map ADT </h3>

   Core functionality:
    <li> M[k] </li>
    <li> M[k]= v </li>
    <li> del M[k] </li>
    <li> len(M) </li>
    <li> iter(M) </li>
    
   Support functions:
    <li> k in M (contains_) </li>
    <li> M.get(k, d=None) </li>
    <li> M.setDefault(k, d) </li>
    <li> M.pop(k, d=None) </li>
    <li> M.popitem() </li>
    <li> M.clear() </li>
    <li> M.items() </li>
    <li> M.keys() </li>
    <li> M.values() </li>
    <li> M==M2 </li>
    <li> M!=M2 </li>
    


<h3> 10.2 Implementation </h3>
<br/>

<strong> Python's MutableMapping Abstract Base Class </strong>

<strong> Mapping- </strong> all nonmutating methods

<strong> Mutable mapping-</strong>  all mapping + mutating methods: all except get, set, len, del, iter

<br/>

<strong>Hierarchy of Mapping ADT implementation class</strong>
<pre>
Base class- MutableMapping

class MutableMapping(Map):
class MapBase(MutableMapping):
      class UnsortedTableMap(MapBase)
      class SortedTableMap(MapBase)
      class HashMapBase(MapBase)
              class ChainHashMap(HashMapBase)
              class ProbeHashMap(HashMapBase)
              
 </pre>             
       
    
    
    

In [61]:
from collections import MutableMapping

class MapBase(MutableMapping):
    class _Item:
        __slots= '_key', '_value'
        
        def __init__(self, k, v):
            self._key=k
            self._value= v

        def __eq__(self, other):
            return self._key== other._key

        def __ne__(self, other):
            return not(self==other)

        def __lt__(self, other):
            return self._key < other._key

In [62]:
class UnsortedTableMap(MapBase):
    def __init__(self, init=None):
        self._table= []
        if(init):
            for i in init:
                self._table.append(self._Item(i[0],i[1]))
                
    def __getitem__(self, k):
        for a in self._table:
            if a._key == k:
                return (a._key, a._value)
        raise KeyError('KeyError')
        
    def __setitem__(self, k,v):
        for a in self._table:
            if a._key== k:
                a._value= v
                return
        self._table.append(self._Item(k,v))
        
    def __delitem__(self, k):
        for a in range(len(self._table)):
            if self._table[a]._key == k:
                self._table.pop(a)
                return
        raise KeyError('keyError')
        
    def __len__(self):
        return len(self._table)
    
    def __iter__(self):
        for i in self._table:
            yield i._key

In [63]:
n= [['A',10], ['B',5], ['C', 8]]
u= UnsortedTableMap(init=n)
u['A']=90

<h3> 10.3: Hash Tables </h3>

<pre>
Hash function: map key k into a respective array bucket in the hash table
    comprises of: Hash code and Compression function:
        Hash code: map key into integer
        Compression function: map Hash code into the range of the hash table [0, N-1]
    handles collision by: Separate Chaining and Open Addressing
        Separate chaining: bucket contains a seconday container (array) to store multiple keys
                           Load factos is the expected size of bucket: n/N where N= number of buckets
                           should be below 1
        Open Addressing: Linear probing/ Quadratic probing to try probing the immediate next slot until empty
                         pros and cons- no auxillary structure but suffers from clustering/ overlaps
                         can use double hashing to avoid collision
      
</pre>

<strong> Hash Code: </strong>
 <ol> 1. Treating the bit represenatation as integer, convert 64 bits into python 32 bit hash by AND/ XOR of 64bits</ol>
 <ol> 2. Polynomial hash, useful for string. take into consideration of position so 'hash01' not equal 'hash10'</ol>
 
<ol>3. Cyclic shift </ol>
Note: can only hash immutable data types such as int, float, str, tuple, frozenset
Instances of user define class is unhashable but can use _hash_ to overwrite


<br/>
<strong> Compression functions: </strong>
<ul> division: i mod N </ul>
<ul> MAD: [(ai+b) mod p ] mod N where p is a prime > N </ul>
<br/>
<br/>

<strong> Analysis </strong>
<pre>
         len(M) : O(1)
         k in M : O(1)
         M[k]=v : O(1)
       del M[k] : O(1)
      findlt,gt : O(logN)
           iter : O(n)
       
</pre>


In [69]:
class HashMapBase(MapBase):
    def __init__(self, cap=11, p=109345121):
        self._table= cap * [None]
        self._n=0
        self._prime= p
        self._shift= randrange(p)
        self._scale= 1+ randrange(p-1)
        
    def _hash_function(self,k):
        return (hash(k)*self._scale + shelf._shift) % self._prime % len(self._table)
    
    def __len__(self):
        return len(self._table)
    
    def __getitem__(self, k):
        j= self._hash_function(k)
        return self._bucket_getitem(j,k)
    
    def __setitem__(self,k, v):
        j= self._hash_function(k)
        self._bucket_setitem(j,k,v)
        if self._n > len(self._table)//2:
            self._resize(2* len(self._table)-1)
    
    def __delitem__(self, k):
        j= self._hash_function(k)
        self._bucket_delitem(j,k)
        self._n-=1
        
    def _resize(self, n):
        old= list(self.items())
        self._table= n* [None]
        self._n=0
        for (k,v) in old:
            self._table[k]= v
            

In [70]:
class ChainHashMap(HashMapBase):
    def _bucket_getitem(self, j, k):
        bucket= self._table[j]
        return bucket[k]
    
    def _bucket_delitem(self, j,k):
        bucket= self._table[j]
        del bucket[k]
        
    def _bucket_setitem(self,j,k,v):
        if self._table[j] is None:
            self._table[j]= UnsortedTableMap()
        oldsize= len(self._table[j])
        self._table[j][k]= v
        if len(self._table[j]) > oldsize:
            size._n+=1

In [72]:
class LinearProbing(HashMapBase):
    
    _AVAIL= object()
    def _isAvailable(self,j):
        return self._table[j] is None or self._table[j] is LinearProbing._AVAIL
    
    def _find_slot(self,j,k):
        firstAvail=None
        
        while True:
            if self._isAvailable(self,j):
                if firstAvail is None:
                    firstAvail= j
                if self._table[j] is None:
                    return(False, firstAvail)
                
            elif k== self._table[j]._key:
                return (True, j)
                
                
            j=(j+1)%len(self._table)
        
    
    def _bucket_getitem(self,j,k):
        found, s= self._find_slot(j,k)
        if not found: return False
        return self._table[s]._value
    
    def _bucket_setitem(self, j, k, v):
        found, s= self._find_slot(j,k)
        if not found:
            self._table[s]= self._Item(k,v)
        else:
            self._table[s]._value= v
    
    def _bucket_delitem(self,j,k):
        found, s= self._find_slot(j,k)
        if not found: return false
        self._table[s] = LinearProbing._AVAIL
        return

<h3> 10.4 Sorted Maps </h3>


<strong> ADT: </strong>
<pre>
    M.findMin()
    M.findMax()
    M.find_lt(k)
    M.find_gt(k)
    M.find_ge(k)
    M.findRange(start,stop)
    iter(M)
    reversed(M)
</pre>
    
    
<strong> Implementation: </strong>
use binary search to find position then can handle all subsequent actions

<br/>
<strong> Analysis </strong>
<pre>
         len(M) : O(1)
         k in M : O(logN)
         M[k]=v : O(logN)
       del M[k] : O(logN)
   findMin, Max : O(1)
      findlt,gt : O(logN)
 reversed, iter : O(n)
       
</pre>


In [65]:
class SortedMap(MapBase):
    def _findIndex(self, k, low, high):
        if high<low:
            return high+1
        else:
            mid= (low+hig)//2
            if k==self._table[mid]._key:
                return mid
            elif k < self._table[mid]._key:
                return self._findIndex(k, low, mid-1)
            else:
                return self._findIndex(k, mid+1, high)
    def __init__(self):
        self._table=[]
        
    def __len__(self):
        return len(self._table)
    
    def __getitem__(self,k):
        j= self._findIndex(k, 0, len(self._table)-1)
        if found:
            return self._table[j]._value
        
        raise KeyError
        
    def __setitem__(self,k,v):
        j= self._findIndex(k, 0, len(self._table)-1)
        if found:
            self._table[j]=v
        else:
            self._table.insert(j, self._Item(k,v))
            
    def __delitem__(self,k):
        j= self._findIndex(k, 0, len(self._table)-1)
        if k==self._table[j]._key:
            self._table.pop(j)
        else:
            raise KeyError
    
    def find_min(self):
        return self._table[0]
    
    def find_gt(self,k):
        j= self._findIndex(k, 0, len(self._table)-1)
        if j<len(self._table) and self._table[j]._key==k:
            return self._table[j+1]._key
            
        